# < Basic Recommendation Algorithm >

---

In [3]:
import matplotlib.pyplot as plt
from matplotlib.image import imread

In [17]:
def show_img(path):
    img = imread(path)

    plt.imshow(img)
    plt.axis('off')  ## grid 없애기
    plt.show()

## 1. Popularity, High Rated Based

* 가장 단순함 => 평점이 높은 것을 추천
* 모두에게 같은 item 추천

In [4]:
ratings = {
     'doky':{'Anne_with_an_E':5,'Strange_Things':3,'The_Escape':3},
     'steve':{'Anne_with_an_E':5,'Strange_Things':1,'The_Escape':4},
     'jamie':{'Anne_with_an_E':0,'Strange_Things':4,'The_Escape':5},
     'bobby':{'Anne_with_an_E':2,'Strange_Things':1,'The_Escape':5}
}

In [5]:
recmd_movielist = dict()

# (A)
for name in ratings:
    for title in ratings[name]:
        score = ratings[name][title]
        
        if title not in recmd_movielist:
            recmd_movielist[title] = score
        else:
            recmd_movielist[title] += score

#



# (B)
for recmd_movietitle in recmd_movielist:
    recmd_movielist[recmd_movietitle] /= 4
#



# (C)
from operator import itemgetter
recmd_movielist = sorted(recmd_movielist.items(), reverse=True, key=itemgetter(1))
#

recmd_movielist

[('The_Escape', 4.25), ('Anne_with_an_E', 3.0), ('Strange_Things', 2.25)]

> 네 사람이 같은 영화에 준 평점들을 각각 더한 다음`(A)`<br>4로 나눠서 각 영화가 받은 평점의 평균을 낸다`(B)`<br>그 평균으로 내림차순 정렬해서 추천 리스트를 만든다`(C)`

---

## 2. Collaborative Filtering

* 사용자가 입력한 선호도(평점)를 사용하여 사용자-항목 선호도(평점) 행렬 계산
* 그 행렬을 사용하여 사용자들 간의 유사도를 계산
* 사용자들 간의 유사도를 바탕으로 모든 항목에 대해 예측 값을 계산하고, 높은 예측 값을 갖는 상위 N개의 추천 목록을 생성

In [6]:
ratings = {
     'doky':{'Anne_with_an_E':5,'Strange_Things':3,'The_Escape':3},
     'steve':{'Anne_with_an_E':5,'Strange_Things':1,'The_Escape':4},
     'jamie':{'Strange_Things':4,'The_Escape':5},
     'bobby':{'Anne_with_an_E':2,'Strange_Things':1,'The_Escape':5}
}

In [7]:
import math

def sim_w_dist(i, j):
    return math.sqrt(pow(i,2) + pow(j,2))

> 피타고라스 공식을 통한 거리 계산으로 유사도 측정

In [8]:
max_sim_w_dist = sim_w_dist(5, 5)
min_sim_w_dist = sim_w_dist(0, 0)
print(min_sim_w_dist, ',', max_sim_w_dist)

0.0 , 7.0710678118654755


> sim_w_dist로 구한 sim의<br>
최대값=루트50, 최소값=0

In [9]:
for name in ratings:
    if name != 'jamie':
        v1 = ratings['jamie']['Strange_Things'] - ratings[name]['Strange_Things']
        v2 = ratings['jamie']['The_Escape'] - ratings[name]['The_Escape']
        sim = sim_w_dist(v1, v2)
        norm_sim = (sim-min_sim_w_dist)/(max_sim_w_dist-min_sim_w_dist)
        print('similarity with', name, ':', sim, '->', norm_sim)

similarity with doky : 2.23606797749979 -> 0.31622776601683794
similarity with steve : 3.1622776601683795 -> 0.4472135954999579
similarity with bobby : 3.0 -> 0.4242640687119285


> jamie가 평가한 Strange_Things, The_Escape를 모두 평가한 사용자와의 유사도 구하기<btr>
* sim_w_dist 함수 사용
* sim을 0과 1 사이로 정규화한 norm_sim 계산 => 1에 가까운 값일수록 유사도 높음

---

## 3. Similarity 계산 방식

1) 평균제곱차이 유사도 (Mean Squared Difference Similarity)

> User-based Collaborative Filter
![image](formula/3_of_similarity/msdSim_userCF.png)
    - Iuv는 사용자 u와 사용자 v 모두에 의해 평가된 상품의 집합
    - |Iuv|는 사용자 u와 사용자 v 모두에 의해 평가된 상품의 수
    - 동일한 조건은 item값

<br>

> Item-based Collaborative Filter
![image](formula/3_of_similarity/msdSim_itemCF.png)
    - Uij는 상품 i와 상품 j 모두를 평가한 사용자의 집합이고 |Uij|는 상품 i와 상품 j 모두를 평가한 사용자의 수
    - 동일한 조건은 user값
    
<br>

> Mean Squared Difference Similarity
![image](formula/3_of_similarity/msdSim_sim.png)
    - Mean Squared Difference(msd)의 역수 -> msd가 클 수록 Similarity 값은 작아짐
    - MSD가 0이 되는 경우를 대응하기 위해 분모에 1을 무조건 더함

> 구현

In [94]:
def sim_msd(data, name1, name2):
    
    diff = 0
    count = 0
    
    for title in data[name1]:
        
        if title in data[name2]:
            ##print('commonly watched movie ->', title)
            diff += pow(data[name1][title] - data[name2][title], 2)
            count += 1
            
    sim_msd = 1/( 1 + (diff/count) )
    
    ##print('** msd similariy = ', sim_msd)
    
    return sim_msd

In [86]:
ratings = {
     'doky':{'Anne_with_an_E':5,'Strange_Things':3,'The_Escape':3},
     'steve':{'Anne_with_an_E':5,'Strange_Things':1,'The_Escape':4},
     'jamie':{'Anne_with_an_E':0,'Strange_Things':4,'The_Escape':5},
     'bobby':{'Anne_with_an_E':2,'Strange_Things':1,'The_Escape':5},
     'cookie':{'Anne_with_an_E':2,'Strange_Things':1}
}

In [87]:
sim_msd(ratings, 'doky', 'cookie')

commonly watched movie -> Anne_with_an_E
commonly watched movie -> Strange_Things


0.13333333333333333

2) 코사인 유사도 (Cosine Similarity)

> 값의 범위 : -1 ~ 1
![image](formula/3_of_similarity/cos.png)
ㄴ> 위의 공식 이용하여 도출

<br>

> 사용자 u와 사용자 v간의 Cosine Similarity
![image](formula/3_of_similarity/cosSim_user.png)
    - 두 사용자가 모두 평가한 상품의 평점을 사용해서 계산
    - 분모는 내적값을 구하는 것과 같음
    - 분자는 벡터의 크기를 구하는 것과 같음

<br>

> 상품 i와 상품 j간의 Cosine Similarity
![image](formula/3_of_similarity/cosSim_item.png)
    - 두 상품의 평점을 사용해서 계산
    - 분모는 내적값을 구하는 것과 같음
    - 분자는 벡터의 크기를 구하는 것과 같음
    
<br>

> 구현

In [88]:
import math
def sim_cosine(data, name1, name2):
    vsize_name1 = 0
    vsize_name2 = 0
    product_sum = 0
    
    for title in data[name1]:
        if title in data[name2]:
            vsize_name1 += pow(data[name1][title], 2)
            vsize_name2 += pow(data[name2][title], 2)
            
            product_sum += data[name1][title]*data[name2][title]
            
    sim_cosine = product_sum/(math.sqrt(vsize_name1)*math.sqrt(vsize_name2))
    
    ##print(sim_cosine)
    
    return sim_cosine

In [89]:
sim_cosine(ratings, 'doky', 'cookie')

0.9970544855015815

3) 피어슨 유사도 (Pearson Similarity)

> 값의 범위 -1 ~ 1<br>
특정인물의 점수기준이 극단적으로 너무 낮거나 높을 경우 유사도에 영향을 크게 주기 때문에, 이를 막기 위해 상관계수 사용

<br>

> 사용자 u와 사용자 v간의 Pearson Similarity
![image](formula/3_of_similarity/pearsonSim_user.png)
    - μu는 사용자 u의 평균 평점

<br>

> 상품 i와 상품 j간의 Pearson Similarity
![image](formula/3_of_similarity/pearsonSim_item.png)
    - μi는 상품 i의 평균 평점
    
<br>

> 구현

In [90]:
def sim_pearson(data, name1, name2):
    sum_name1 = 0
    sum_name2 = 0
    count = 0
    
    for title in data[name1]:
        if title in data[name2]:
            sum_name1 += data[name1][title]
            sum_name2 += data[name2][title]
            count += 1
            
    avg_name1 = sum_name1/count
    avg_name2 = sum_name2/count
    
    vsize_name1 = vsize_name2 = product_sum = 0
    
    for title in data[name1]:
        if title in data[name2]:
            vsize_name1 += pow(data[name1][title]-avg_name1, 2)
            vsize_name2 += pow(data[name2][title]-avg_name2, 2)
            product_sum += (data[name1][title]-avg_name1)*(data[name2][title]-avg_name2)
            
    sim_pearson = product_sum / (math.sqrt(vsize_name1)*math.sqrt(vsize_name2))
    ## print(sim_pearson)
    return sim_pearson

In [91]:
sim_pearson(ratings, 'doky', 'cookie')

0.9999999999999998

---
## 상위 N개 추천

> 구현

In [103]:
def top_match(data, name, n=3, sim_function=sim_pearson):
    top_list = []
    for user in data:
        if user != name:
            top_list.append((sim_function(data, name, user), user))  # ({유사도}, {사람})
    top_list.sort(reverse=True)
    
    topN_list = top_list[:n]
    
    for idx, user in enumerate(topN_list):
        print(idx+1, '등 : ', user)
    
    return topN_list

In [101]:
print('<msd>')
top_match(ratings, 'doky', 3, sim_msd)
print('-'*50)
print('<cosine>')
top_match(ratings, 'doky', 3, sim_cosine)
print('-'*50)
print('<pearson>')
top_match(ratings, 'doky', 3, sim_pearson)

<msd>
1 등 :  (0.37499999999999994, 'steve')
2 등 :  (0.15, 'bobby')
3 등 :  (0.13333333333333333, 'cookie')
--------------------------------------------------
<cosine>
1 등 :  (0.9970544855015815, 'cookie')
2 등 :  (0.9412416106700233, 'steve')
3 등 :  (0.7795844649455863, 'bobby')
--------------------------------------------------
<pearson>
1 등 :  (0.9999999999999998, 'cookie')
2 등 :  (0.6933752452815364, 'steve')
3 등 :  (-0.2773500981126146, 'bobby')


[(0.9999999999999998, 'cookie'),
 (0.6933752452815364, 'steve'),
 (-0.2773500981126146, 'bobby')]

---
## 유사도와 KNN을 활용한 예측 값 계산 및 추천 목록 생성 기법
* 사용자들 간의 유사도를 이용하여 모든 아이템에 대한 예측 평점을 계산하고, 높은 값을 갖는 상위 N개의 추천 목록 생성

<br>

* K Nearest Neighbors(KNN) 가중치 예측 기법
    * 사용자(또는 아이템)와 유사도가 큰 k개의 사용자(또는 아이템) 벡터를 사용해 가중 평균을 구해서 가중치를 예측

> KNN Basic
![image](formula/KNN/KNN_Basic.png)
* 평점들을 단순히 가중 평균
* Nk는 유사도가 큰 벡터 상위 k개의 집합

> 구현

In [102]:
ratings_expand = {
    '마동석': {
        '택시운전사': 3.5,
        '남한산성': 1.5,
        '킹스맨:골든서클': 3.0,
        '범죄도시': 3.5,
        '아이 캔 스피크': 2.5,
        '꾼': 3.0,
    },
    '이정재': {
        '택시운전사': 5.0,
        '남한산성': 4.5,
        '킹스맨:골든서클': 0.5,
        '범죄도시': 1.5,
        '아이 캔 스피크': 4.5,
        '꾼': 5.0,
    },
    '윤계상': {
        '택시운전사': 3.0,
        '남한산성': 2.5,
        '킹스맨:골든서클': 1.5,
        '범죄도시': 3.0,
        '꾼': 3.0,
        '아이 캔 스피크': 3.5,
    },
    '설경구': {
        '택시운전사': 2.5,
        '남한산성': 3.0,
        '범죄도시': 4.5,
        '꾼': 4.0,
    },
    '최홍만': {
        '남한산성': 4.5,
        '킹스맨:골든서클': 3.0,
        '꾼': 4.5,
        '범죄도시': 3.0,
        '아이 캔 스피크': 2.5,
    },
    '홍수환': {
        '택시운전사': 3.0,
        '남한산성': 4.0,
        '킹스맨:골든서클': 1.0,
        '범죄도시': 3.0,
        '꾼': 3.5,
        '아이 캔 스피크': 2.0,
    },
    '나원탁': {
        '택시운전사': 3.0,
        '남한산성': 4.0,
        '꾼': 3.0,
        '범죄도시': 5.0,
        '아이 캔 스피크': 3.5,
    },
    '소이현': {
        '남한산성': 4.5, 
        '아이 캔 스피크': 1.0,
        '범죄도시': 4.0
    }
}

In [169]:
def get_KNN_recommend(data, name, n=3, sim_function=sim_pearson):
    
    result = top_match(data, name, n, sim_function)
    
    score_dic = dict()
    sim_dic = dict()
    li = []
    
    for sim, user in result:
        print(sim, user)
        if sim<0: continue
        for title in data[user]:
            if title not in data[name]:
                score = sim * data[user][title]
                score_dic.setdefault(title, 0)
                score_dic[title] += score
                
                sim_dic.setdefault(title, 0)
                sim_dic[title] += sim
                
    for title in score_dic:
        score_dic[title] /= sim_dic[title]
        li.append((score_dic[title], title)) # (예측평점, 영화제목)
        
    li.sort(reverse=True)
    
    print('\n** Recommendation')
    for rating, score in enumerate(li):
        print(rating+1, '등: ', score[1], ', ', score[0], '점')
    
    return li

In [170]:
get_KNN_recommend(ratings_expand, '소이현')

1 등 :  (0.924473451641905, '홍수환')
2 등 :  (0.7824666287099552, '최홍만')
3 등 :  (0.6628489803598702, '나원탁')
0.924473451641905 홍수환
0.7824666287099552 최홍만
0.6628489803598702 나원탁

** Recommendation
1 등:  꾼 ,  3.6903300787431923 점
2 등:  택시운전사 ,  3.0 점
3 등:  킹스맨:골든서클 ,  1.9168062051113846 점


[(3.6903300787431923, '꾼'), (3.0, '택시운전사'), (1.9168062051113846, '킹스맨:골든서클')]

> KNN with Means
![image](formula/KNN/KNN_means.png)
* 평점들을 평균값 기준으로 가중 평균

<br>

> 구현

In [172]:
sum = 0
count = 0

for name in ratings_expand:
    for title in ratings_expand[name]:
        sum += ratings_expand[name][title]
        count += 1
    
    ratings_expand[name]['avg'] = sum/count
    
ratings_expand

{'마동석': {'택시운전사': 3.5,
  '남한산성': 1.5,
  '킹스맨:골든서클': 3.0,
  '범죄도시': 3.5,
  '아이 캔 스피크': 2.5,
  '꾼': 3.0,
  'avg': 2.8333333333333335},
 '이정재': {'택시운전사': 5.0,
  '남한산성': 4.5,
  '킹스맨:골든서클': 0.5,
  '범죄도시': 1.5,
  '아이 캔 스피크': 4.5,
  '꾼': 5.0,
  'avg': 3.1666666666666665},
 '윤계상': {'택시운전사': 3.0,
  '남한산성': 2.5,
  '킹스맨:골든서클': 1.5,
  '범죄도시': 3.0,
  '꾼': 3.0,
  '아이 캔 스피크': 3.5,
  'avg': 3.0277777777777777},
 '설경구': {'택시운전사': 2.5,
  '남한산성': 3.0,
  '범죄도시': 4.5,
  '꾼': 4.0,
  'avg': 3.1136363636363638},
 '최홍만': {'남한산성': 4.5,
  '킹스맨:골든서클': 3.0,
  '꾼': 4.5,
  '범죄도시': 3.0,
  '아이 캔 스피크': 2.5,
  'avg': 3.185185185185185},
 '홍수환': {'택시운전사': 3.0,
  '남한산성': 4.0,
  '킹스맨:골든서클': 1.0,
  '범죄도시': 3.0,
  '꾼': 3.5,
  '아이 캔 스피크': 2.0,
  'avg': 3.106060606060606},
 '나원탁': {'택시운전사': 3.0,
  '남한산성': 4.0,
  '꾼': 3.0,
  '범죄도시': 5.0,
  '아이 캔 스피크': 3.5,
  'avg': 3.1842105263157894},
 '소이현': {'남한산성': 4.5, '아이 캔 스피크': 1.0, '범죄도시': 4.0, 'avg': 3.182926829268293}}

In [174]:
def get_KNNmeans_recommend(data, name, n=3, sim_function=sim_pearson):
    
    result = top_match(data, name, n, sim_function)
    
    score_dic = dict()
    sim_dic = dict()
    li = []
    
    for sim, user in result:
        print(sim, user)
        if sim<0: continue
        for title in data[user]:
            if title not in data[name]:
                score = sim * (data[user][title]-data[user]['avg'])
                score_dic.setdefault(title, 0)
                score_dic[title] += score
                
                sim_dic.setdefault(title, 0)
                sim_dic[title] += sim
                
    for title in score_dic:
        score_dic[title] = data[name]['avg']+(score_dic[title]/sim_dic[title])
        li.append((score_dic[title], title)) # (예측평점, 영화제목)
        
    li.sort(reverse=True)
    
    print('\n** Recommendation')
    for rating, score in enumerate(li):
        print(rating+1, '등: ', score[1], ', ', score[0], '점')
    
    return li

In [175]:
get_KNNmeans_recommend(ratings_expand, '소이현')

1 등 :  (0.9228578895516366, '홍수환')
2 등 :  (0.7790435921243095, '최홍만')
3 등 :  (0.517441118049583, '나원탁')
0.9228578895516366 홍수환
0.7790435921243095 최홍만
0.517441118049583 나원탁

** Recommendation
1 등:  꾼 ,  3.765320026911711 점
2 등:  택시운전사 ,  3.0487901210241017 점
3 등:  킹스맨:골든서클 ,  1.9561449029750133 점


[(3.765320026911711, '꾼'),
 (3.0487901210241017, '택시운전사'),
 (1.9561449029750133, '킹스맨:골든서클')]

---

# Reference

* https://www.fun-coding.org/recommend_basic2.html